<a href="https://colab.research.google.com/github/diisilva/simex_plugin/blob/main/AI01_BRASIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pré-processamento das imagens Landsat**
Lista de imagens Landsat filtradas L1T SR no Earth Engine.


##Importação de Bilbiotecas


In [ ]:
# Importar biblioteca Google Earth Engine (GEE)
import ee
# Importa biblioteca Pandas
import pandas as pd

##Autenticações

In [ ]:
# Autenticação GEE - necessário ter uma conta no code editor
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Gr-pvO20WKuyMWdFgfQvvIeu47z2aLyoxl4m1mit53g&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6Vs5ZBwPj_S5khKtIu7wXMgPml7OplQ3AmI69RnEYS21Tx7WSPPbo

Successfully saved authorization token.


In [ ]:
# Autenticação Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Inicializar biblioteca do GEE
ee.Initialize()

# Funções de processamento

In [ ]:
 # Normalizar propriedades de satélites Landsat 5, 7 e 8 
 def setProperties(image):
      
            cloudCover = ee.Algorithms.If(image.get('CLOUD_COVER'),\
                    image.get('CLOUD_COVER'),\
                    image.get('CLOUDY_PIXEL_PERCENTAGE'))
                
            date = ee.Algorithms.If(image.get('DATE_ACQUIRED'),image.get('DATE_ACQUIRED'),ee.Algorithms.If(image.get('SENSING_TIME'),image.get('SENSING_TIME'),image.get('GENERATION_TIME')))
            
            satellite = ee.Algorithms.If(image.get('SPACECRAFT_ID'),\
                    image.get('SPACECRAFT_ID'),\
                    ee.Algorithms.If(image.get('SATELLITE'),\
                        image.get('SATELLITE'),\
                        image.get('SPACECRAFT_NAME')\
                    )\
                )
            
            azimuth = ee.Algorithms.If(image.get('SUN_AZIMUTH'),\
                    image.get('SUN_AZIMUTH'),\
                    ee.Algorithms.If(image.get('SOLAR_AZIMUTH_\ANGLE'),\
                        image.get('SOLAR_AZIMUTH_ANGLE'),
                        image.get('MEAN_SOLAR_AZIMUTH_ANGLE')\
                    )\
                )
            
            elevation = ee.Algorithms.If(image.get('SUN_ELEVATION'),\
                    image.get('SUN_ELEVATION'),\
                    ee.Algorithms.If(image.get('SOLAR_ZENITH_ANGLE'),\
                        ee.Number(90).subtract(image.get('SOLAR_ZENITH_ANGLE')),\
                        ee.Number(90).subtract(image.get('MEAN_SOLAR_ZENITH_ANGLE'))\
                    )\
                )
            
            reflectance = ee.Algorithms.If(
                    ee.String(ee.Dictionary(ee.Algorithms.Describe(image)).get('id')).match('SR').length(),\
                    'SR',\
                    'TOA'\
                )
            
            return image\
                    .set('cloud_cover', cloudCover)\
                    .set('satellite_name', satellite)\
                    .set('sun_azimuth_angle', azimuth)\
                    .set('sun_elevation_angle', elevation)\
                    .set('reflectance', reflectance)\
                    .set('date', ee.Date(date).format('Y-MM-dd'))

In [ ]:
# obter coleção de imagens Landsat 5, 7 e 8
def getCollection(obj):
    
            collection = ee.ImageCollection(obj['collectionid']).filterBounds(obj['geometry']).filterDate(ee.Date(obj['dateStart']),ee.Date(obj['dateEnd'])).map(setProperties)
      
            collection = collection.filterMetadata('cloud_cover','less_than',obj['cloud_cover'])

            return collection

In [ ]:
# normalizar nomes da bandas de satélites Landsat 5, 7 e 8
def bandNames(key):
    
            bandNames = {'sentinel2' : {
                'bandNames': ['B2','B3','B4','B8','B10','B11','B12','QA60'],
                'newNames': ['blue','green','red','nir','cirrus','swir1','swir2','QA60'] },
                        'sentinel2_SR' : {
                  'bandNames': ['B2','B3','B4','B8','B11','B12','QA60'],
                  'newNames': ['blue','green','red','nir','swir1','swir2','QA60']
                }, 'l5': {
      'bandNames': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa', 'B6'],
      'newNames': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa', 'temp']
  },
  
 'l7': {
      'bandNames': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa', 'B6'],
      'newNames': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa', 'temp']
  },

  'l8' : {
      'bandNames': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'pixel_qa', 'B11'],
      'newNames': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal', 'pixel_qa', 'temp']
  }}
        
            return bandNames[key]

In [ ]:
# Exportação de CSV para o google drive
def exportTabletoDrive(featureCollection, description,tableName):
  task = ee.batch.Export.table.toDrive(
    collection = featureCollection,
    description = tableName,
    fileNamePrefix = tableName,
    fileFormat = 'CSV',
  )
  task.start()   

# Paramêtros de entrada

In [ ]:
# Parametros de entrada
area = 1
name = 'AI0' + str(area) + '_BRASIL'
dataInicial = '1985-01-01'
dataFinal = '2021-01-01'
coberturaNuvem = 50
assetAreas = ee.FeatureCollection('users/diimanut/ITV/ALL_GRIDS_VALE')

# Processamento

In [ ]:
#filtro de geometria para a área de entrada setada nas Opções de área
geometria = assetAreas.filterMetadata('AI','equals',area)
#Gerando coleção Landsat 8
parametrosColecaoL8 = {
    'collectionid': 'LANDSAT/LC08/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('2013-04-11'),
    'dateEnd': ee.Date(dataFinal),
    'cloud_cover': coberturaNuvem,
}
colecaoL8 = getCollection(parametrosColecaoL8).select(bandNames("l8")['bandNames'],bandNames("l8")['newNames'])
#Gerando coleção Landsat 7
parametrosColecaoL7 = {
    'collectionid': 'LANDSAT/LE07/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('1999-01-01'),
    'dateEnd': ee.Date('2013-04-11'),
    'cloud_cover': coberturaNuvem,
}
colecaoL7 = getCollection(parametrosColecaoL7).select(bandNames("l7")['bandNames'],bandNames("l7")['newNames'])
#Gerando coleção Landsat 5
parametrosColecaoL5 = {
    'collectionid': 'LANDSAT/LT05/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('1985-01-01'),
    'dateEnd': ee.Date('2013-04-11'),
    'cloud_cover': coberturaNuvem,
}

colecaoL5 = getCollection(parametrosColecaoL5).select(bandNames("l5")['bandNames'],bandNames("l5")['newNames'])
#Após normalizar os nomes dos metadados e renomear as bandas,
#junta-se as coleções Landsat 5,7 e 8 numa coleção total
colTotal = colecaoL5.merge(colecaoL7).merge(colecaoL8)

geometria = assetAreas.filterMetadata('AI','equals',area)
parametrosColecaoL8 = {
    'collectionid': 'LANDSAT/LC08/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('2013-04-11'),
    'dateEnd': ee.Date(dataFinal),
    'cloud_cover': coberturaNuvem,
}
colecaoL8 = getCollection(parametrosColecaoL8).select(bandNames("l8")['bandNames'],bandNames("l8")['newNames'])
#print('colecaoL8 size\n',colecaoL8.size().getInfo())

parametrosColecaoL7 = {
    'collectionid': 'LANDSAT/LE07/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('1999-01-01'),
    'dateEnd': ee.Date('2013-04-11'),
    'cloud_cover': coberturaNuvem,
}
colecaoL7 = getCollection(parametrosColecaoL7).select(bandNames("l7")['bandNames'],bandNames("l7")['newNames'])
#print('colecaoL7 size\n',colecaoL7.size().getInfo())

parametrosColecaoL5 = {
    'collectionid': 'LANDSAT/LT05/C01/T1_SR',
    'geometry': geometria,
    'dateStart': ee.Date('1985-01-01'),
    'dateEnd': ee.Date('2013-04-11'),
    'cloud_cover': coberturaNuvem,
}

colecaoL5 = getCollection(parametrosColecaoL5).select(bandNames("l5")['bandNames'],bandNames("l5")['newNames'])
#print('colecaoL5 size\n',colecaoL5.size().getInfo())

colTotal = colecaoL5.merge(colecaoL7).merge(colecaoL8)


In [ ]:
featureCollection = colTotal.map(lambda img: ee.Feature(None,ee.Dictionary({
            'satellite_name':img.get('satellite_name'),
            'WRS_ROW':img.get('WRS_ROW'),
            'WRS_PATH':img.get('WRS_PATH'),
            'LANDSAT_ID':img.get('LANDSAT_ID'),
            'date':img.get('date'),
            'cloud_cover':img.get('cloud_cover'),
            'reflectance':img.get('reflectance'),
            'mosaic': geometria.first().get('Mosaico'),
             'country': geometria.first().get('Pais'),
             'day_of_year':img.date().getRelative('day', 'year')
        })))


In [ ]:
#Exportação do CSV
exportTabletoDrive(featureCollection, name,name)